<a href="https://colab.research.google.com/github/CalvHobbes/mountain-classifier/blob/main/ViT_FineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install transformers pip comet_ml datasets evaluate


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.5/101.5 kB 5.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 709.7/709.7 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.8/981.8 kB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 103.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 1

In [2]:
!git clone https://github.com/CalvHobbes/mountain-classifier.git

Cloning into 'mountain-classifier'...
remote: Enumerating objects: 717, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 717 (delta 3), reused 3 (delta 0), pack-reused 705 (from 1)
Receiving objects: 100% (717/717), 228.65 MiB | 32.55 MiB/s, done.
Resolving deltas: 100% (3/3), done.
Updating files: 100% (680/680), done.


In [3]:
import comet_ml
from comet_ml import Experiment
from comet_ml import start
from comet_ml.integration.pytorch import log_model
from google.colab import userdata
import os

experiment = start(
  api_key="tv9OhbrvWqXeyrnIlTKVBUkwQ",
  project_name="mountain-classifier",
  workspace="calvhobbes"
)

comet_key = userdata.get("COMET_API_KEY")
comet_ml.login(project_name="mountain-classifier", api_key=comet_key)

os.environ["COMET_LOG_ASSETS"] = "True"
os.environ["COMET_API_KEY"] = comet_key
os.environ["WANDB_DISABLED"] = "true"

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/calvhobbes/mountain-classifier/c042338139d5421fbb640cf64c68accc

COMET INFO: Couldn't find a Git repository in '/content' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.
COMET INFO: Valid Comet API Key saved in /root/.comet.config (set COMET_CONFIG to change where it is saved).


In [4]:
from PIL import Image
import numpy as np
import torch

def preprocess_image(image):
    image = Image.open(image["image"]).convert("RGB")
    image = image.resize((224, 224))
    image = np.array(image) / 255.0
    image = image.transpose(2, 0, 1)  # Convert to (C, H, W) format
    image = torch.tensor(image)
    return image

In [5]:
from datasets import load_dataset

# Load dataset from directory structure (like `train` and `test` folders)
# This assumes your `train_dir` and `test_dir` folders have subfolders per class
train_dir = "mountain-classifier/data/train"
test_dir = "mountain-classifier/data/test"

# Load the training and test datasets
train_dataset = load_dataset("imagefolder", data_dir=train_dir)
test_dataset = load_dataset("imagefolder", data_dir=test_dir)



Resolving data files:   0%|          | 0/541 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Resolving data files:   0%|          | 0/136 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [6]:
train_dataset

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 541
    })
})

In [7]:
test_dataset

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 136
    })
})

In [8]:
from transformers import ViTFeatureExtractor,AutoModelForImageClassification

model_name_or_path = 'google/vit-base-patch16-224'
feature_extractor = ViTFeatureExtractor.from_pretrained(model_name_or_path)
feature_extractor

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


ViTFeatureExtractor {
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.5,
    0.5,
    0.5
  ],
  "image_processor_type": "ViTFeatureExtractor",
  "image_std": [
    0.5,
    0.5,
    0.5
  ],
  "resample": 2,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 224,
    "width": 224
  }
}

In [9]:
def process_example(example):
    inputs = feature_extractor(example['image'], return_tensors='pt')
    inputs['labels'] = example['label']
    return inputs

In [10]:
process_example(train_dataset['train'][0])

{'pixel_values': tensor([[[[-0.9216, -0.9137, -0.9137,  ..., -0.9137, -0.9137, -0.9137],
          [-0.9137, -0.9137, -0.9137,  ..., -0.9137, -0.9137, -0.9137],
          [-0.9137, -0.9216, -0.9216,  ..., -0.9137, -0.9059, -0.9059],
          ...,
          [-0.1294, -0.0275,  0.0431,  ..., -0.6863, -0.7020, -0.7255],
          [-0.3412, -0.2549, -0.1216,  ..., -0.6000, -0.6706, -0.6627],
          [-0.3882, -0.3098, -0.2941,  ..., -0.4588, -0.6471, -0.7020]],

         [[-0.7255, -0.7176, -0.7176,  ..., -0.7725, -0.7725, -0.7725],
          [-0.7176, -0.7176, -0.7176,  ..., -0.7725, -0.7725, -0.7725],
          [-0.7176, -0.7255, -0.7255,  ..., -0.7725, -0.7647, -0.7647],
          ...,
          [-0.1686, -0.0902,  0.0118,  ..., -0.7255, -0.7569, -0.7804],
          [-0.3725, -0.3098, -0.1451,  ..., -0.6549, -0.7333, -0.7333],
          [-0.4275, -0.3412, -0.3255,  ..., -0.5216, -0.7255, -0.7804]],

         [[-0.2863, -0.2784, -0.2784,  ..., -0.3882, -0.3961, -0.3961],
          [-0

In [11]:
from PIL import Image
import torchvision.transforms as transforms

def transform(example_batch):
    #
    # Take a list of PIL images and turn them to pixel values
    inputs = feature_extractor([x.convert("RGB") for x in example_batch['image']], return_tensors='pt')

    # Don't forget to include the labels!
    inputs['labels'] = example_batch['label']
    return inputs

def transform_old(example_batch):
    # Define the desired image size
    desired_size = (224, 224)

    # Resize the images to the desired size
    resized_images = [transforms.Resize(desired_size)(x.convert("RGB")) for x in example_batch['image']]

    # Convert resized images to pixel values
    inputs = feature_extractor(resized_images, return_tensors='pt')

    # Don't forget to include the labels!
    inputs['label'] = example_batch['label']

    return inputs

prepared_train_ds = train_dataset.with_transform(transform)
prepared_test_ds = test_dataset.with_transform(transform)



In [12]:
prepared_train_ds['train'][0]["pixel_values"].shape

torch.Size([3, 224, 224])

In [13]:
labels = prepared_train_ds['train'].features['label'].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label


In [14]:
import numpy as np
import evaluate

def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['labels'] for x in batch])
    }

metric = evaluate.load("accuracy")

def compute_metrics(p):
    return metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)

In [15]:
from transformers import TrainingArguments
training_args = TrainingArguments(
  output_dir="./vit-base-v1-eval-steps",
  per_device_train_batch_size=16,
  eval_strategy="steps",
  num_train_epochs=4,
  fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=10,
  learning_rate=2e-4,
  save_total_limit=2,
  remove_unused_columns=False,
  push_to_hub=False,
  report_to='none',
  load_best_model_at_end=True,
)

training_args = TrainingArguments(
    output_dir="./vit-base-v1-eval-epoch",
    eval_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=16,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    learning_rate=5e-5,
    weight_decay=0.01,
    warmup_ratio=0.1,
    fp16=True,
    logging_steps=10,
    save_total_limit=1,
    remove_unused_columns=False,
    push_to_hub=False,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    report_to='none'
)


NameError: name 'TrainingArguments' is not defined

In [ ]:
from transformers import ViTForImageClassification

model = ViTForImageClassification.from_pretrained(
    model_name_or_path,
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True
)

In [ ]:
from transformers import Trainer, TrainingArguments
import comet_ml
from comet_ml import Experiment
from comet_ml import start
from comet_ml.integration.pytorch import log_model
from google.colab import userdata
import os


# training_args = TrainingArguments(
#     output_dir="my_awesome_food_model",
#     remove_unused_columns=False,
#     eval_strategy="epoch",
#     save_strategy="epoch",
#     learning_rate=5e-5,
#     per_device_train_batch_size=16,
#     gradient_accumulation_steps=4,
#     per_device_eval_batch_size=16,
#     num_train_epochs=3,
#     warmup_ratio=0.1,
#     logging_steps=10,
#     load_best_model_at_end=True,
#     metric_for_best_model="accuracy",
#     push_to_hub=True,
# )

# Define training arguments - previous
# training_args = TrainingArguments(
#     output_dir="./results",
#     evaluation_strategy="epoch",
#     save_strategy="epoch",
#     per_device_train_batch_size=16,
#     gradient_accumulation_steps=4,
#     per_device_eval_batch_size=16,
#     num_train_epochs=5,
#     learning_rate=5e-5,
#     weight_decay=0.01,
#     warmup_ratio=0.1,
#     fp16=True,
#     logging_steps=10,
#     save_total_limit=1,
#     remove_unused_columns=False,
#     push_to_hub=False,
#     load_best_model_at_end=True,
#     metric_for_best_model="accuracy",
#     report_to=['comet_ml']
# )
experiment.log_parameters(training_args)





In [ ]:
import numpy as np
import evaluate



# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=prepared_train_ds['train'],
    eval_dataset=prepared_test_ds['train'],
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    tokenizer=feature_extractor
)



In [ ]:

train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

In [ ]:
log_model(experiment, model=model, model_name="Vit-14Summit-Classifier")

In [ ]:
metrics = trainer.evaluate(prepared_test_ds['train'])
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

In [ ]:
experiment.end()